In [ ]:
# !pip install numexpr
# !pip install langchain
# !pip install openai
# !pip install openai-whisper
# !pip install sentence-transformers
# !pip install unstructured
# !pip install chromadb

In [1]:
import inspect
import random
import re
from typing import Optional

In [8]:
import os

api_key = "enter_your_api_key"

# Set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = api_key

In [9]:
from langchain import LLMChain
from langchain.chains import LLMChain, LLMMathChain, SequentialChain, TransformChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.tools import Tool

# Langchain 101

We first create a `PromptTemplate` class and initialize a templat

In [10]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# user question
question = "Which NFL team won the Super Bowl in the 2010 season?"

In [11]:
prompt.format(question=question)

'Question: Which NFL team won the Super Bowl in the 2010 season?\n\nAnswer: '

In [12]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [13]:
llm_chain = LLMChain(prompt=prompt, llm=model)

In [14]:
# ask the user question about NFL 2010
print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


Asking multiple questions

In [15]:
qs = [
    {"question": "Which NFL team won the Super Bowl in the 2010 season?"},
    {"question": "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {"question": "Who was the 12th person on the moon?"},
    {"question": "How many eyes does a blade of grass have?"},
]

In [16]:
res = llm_chain.generate(qs)

In [17]:
res.generations[0][0].text.strip()

'The Green Bay Packers won the Super Bowl in the 2010 season.'

In [18]:
res.generations[1][0].text.strip()

'193.04 centimeters'

In [19]:
res.generations[2][0].text.strip()

'The 12th person to walk on the moon was astronaut Alan Bean, who was part of the Apollo 12 mission in November 1969.'

In [20]:
res.generations[3][0].text.strip()

'A blade of grass does not have any eyes.'

## What are chains anyway?

Definition: Chains are one of the fundamental building blocks of this lib (as you can guess!).

The official definition of chains is the following:

>A chain is made up of links, which can be either primitives or other chains. Primitives can be either prompts, llms, utils, or other chains.

So a chain is basically a pipeline that processes an input by using a specific combination of primitives. Intuitively, it can be thought of as a 'step' that performs a certain set of operations on an input and returns the result. They can be anything from a prompt-based pass through a LLM to applying a Python function to an text.

Chains are divided in three types: Utility chains, Generic chains and Combine Documents chains. In this edition, we will focus on the first two since the third is too specific (will be covered in due course).

1. Utility Chains: chains that are usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.
2. Generic Chains: chains that are used as building blocks for other chains but cannot be used out of the box on their own.

### Utility Chains

In [21]:
llm_math = LLMMathChain(llm=model, verbose=True)

d:\PROJECTS\REVIEWED PROJECTS\LLM_2_customer_support\customer_support_new\customer_support\csvn\lib\site-packages\langchain\chains\llm_math\base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [22]:
llm_math.run("What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

Let's see what is going on here. The chain recieved a question in natural language and sent it to the llm. The llm returned a Python code which the chain compiled to give us an answer. A few questions arise.. How did the llm know that we wanted it to return Python code?

#### Enter Prompts

The question we send as input to the chain is not the only input that the llm recieves 😉. The input is inserted into a wider context, which gives precise instructions on how to interpret the input we send. This is called a prompt. Let's see what this chain's prompt is!

In [23]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



Ok.. let's see what we got here. So, we are literally telling the llm that for complex math problems it should not try to do math on its own but rather it should print a Python code that will calculate the math problem instead. Probably, if we just sent the query without any context, the llm would try (and fail) to calculate this on its own. Wait! This is testable.. let's try it out! 🧐

In [24]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=["question"], template="{question}")
llm_chain = LLMChain(prompt=prompt, llm=model)

# we ask the llm for the answer with no context

llm_chain.run("What is 13 raised to the .3432 power?")

'\n\n2.907'

Wrong answer! Herein lies the power of prompting and one of our most important insights so far:

Insight: by using prompts intelligently, we can force the llm to avoid common pitfalls by explicitly and purposefully programming it to behave in a certain way.

### Generic Chains

There are only three Generic Chains in langchain and we will go all in to showcase them all in the same example. Let's go!

Say we have had experience of getting dirty input texts. Specifically, as we know, llms charge us by the number of tokens we use and we are not happy to pay extra when the input has extra characters. Plus its not neat 😉

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [25]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r"(\r\n|\r|\n){2,}", r"\n", text)
    text = re.sub(r"[ \t]+", " ", text)

    return {"output_text": text}

Importantly, when we initialize the chain we do not send an llm as an argument. As you can imagine, not having an llm makes this chain's abilities much weaker than the example we saw earlier. However, as we will see next, combining this chain with other chains can give us highly desirable results.

In [26]:
clean_extra_spaces_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [27]:
clean_extra_spaces_chain.run(
    "A random text  with   some irregular spacing.\n\n\n     Another one   here as well."
)

'A random text with some irregular spacing.\n Another one here as well.'

Great! Now things will get interesting.

Say we want to use our chain to clean an input text and then paraphrase the input in a specific style, say a poet or a policeman. As we now know, the TransformChain does not use a llm so the styling will have to be done elsewhere. That's where our LLMChain comes in. We know about this chain already and we know that we can do cool things with smart prompting so let's take a chance!

In [28]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

And next, initialize our chain:

In [29]:
style_paraphrase_chain = LLMChain(llm=model, prompt=prompt, output_key="final_output")

Great! Notice that the input text in the template is called 'output_text'. Can you guess why?

We are going to pass the output of the `TransformChain` to the `LLMChain`!

In [30]:
sequential_chain = SequentialChain(
    chains=[clean_extra_spaces_chain, style_paraphrase_chain],
    input_variables=["text", "style"],
    output_variables=["final_output"],
)

Our input is the langchain docs description of what chains are but dirty with some extra spaces all around.

In [31]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

We are all set. Time to get creative!

In [32]:
print(sequential_chain.run({"text": input_text, "style": "a poet"}))


Chains bind us, let us join
Components to make one app divine
For instance, take user input, format it with a PromptTemplate
Then pass the response to an LLM
We can make more complex chains, by combining multiple or with other components too.


## Langchain Parsing

In [33]:
""" Define the data structure we want to be parsed out from the LLM response

notice that the class contains a setup (a string) and a punchline (a string.
The descriptions are used to construct the prompt to the llm. This particular
example also has a validator which checks if the setup contains a question mark.

from: https://python.langchain.com/docs/modules/model_io/output_parsers/pydantic
"""


class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [34]:
"""Defining the query from the user
"""
joke_query = "Tell me a joke about parrots"

In [35]:
"""Defining the prompt to the llm

from: https://python.langchain.com/docs/modules/model_io/output_parsers/pydantic
"""
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

print(_input.text)

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
Tell me a joke about parrots



In [36]:
"""Declaring a model and querying it with the parser defined input
"""

output = model(_input.to_string())
parser.parse(output)

Joke(setup="Why don't parrots make good detectives?", punchline="Because they're always repeating themselves!")

# LLMs as Graphs

In [37]:
# defining the model used in this test
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [38]:
class sampleOutputTemplate(BaseModel):
    output: str = Field(description="contact information")

## Utilities

In [39]:
"""Defining utility functions for constructing a readable exchange
"""


def system_output(output):
    """Function for printing out to the user"""
    print("======= Bot =======")
    print(output)


def user_input():
    """Function for getting user input"""
    print("======= Human Input =======")
    return input()


def parsing_info(output):
    """Function for printing out key info"""
    print(f"*Info* {output}")

## Edges

In [40]:
class Edge:

    """Edge
    at its highest level, an edge checks if an input is good, then parses
    data out of that input if it is good
    """

    def __init__(
        self, condition, parse_prompt, parse_class, llm, max_retrys=3, out_node=None
    ):
        """
        condition (str): a True/False question about the input
        parse_query (str): what the parser whould be extracting
        parse_class (Pydantic BaseModel): the structure of the parse
        llm (LangChain LLM): the large language model being used
        """
        self.condition = condition
        self.parse_prompt = parse_prompt
        self.parse_class = parse_class
        self.llm = llm

        # how many times the edge has failed, for any reason, for deciding to skip
        # when successful this resets to 0 for posterity.
        self.num_fails = 0

        # how many retrys are acceptable
        self.max_retrys = max_retrys

        # the node the edge directs towards
        self.out_node = out_node

    def check(self, input):
        """ask the llm if the input satisfies the condition"""
        validation_query = f"following the output schema, does the input satisfy the condition?\ninput:{input}\ncondition:{self.condition}"

        class Validation(BaseModel):
            is_valid: bool = Field(description="if the condition is satisfied")

        parser = PydanticOutputParser(pydantic_object=Validation)
        input = f"Answer the user query.\n{parser.get_format_instructions()}\n{validation_query}\n"
        return parser.parse(self.llm(input)).is_valid

    def parse(self, input):
        """ask the llm to parse the parse_class, based on the parse_prompt, from the input"""
        parse_query = f'{self.parse_prompt}:\n\n"{input}"'
        parser = PydanticOutputParser(pydantic_object=self.parse_class)
        input = f"Answer the user query.\n{parser.get_format_instructions()}\n{parse_query}\n"
        return parser.parse(self.llm(input))

    def execute(self, input):
        """Executes the entire edge
        returns a dictionary:
        {
            continue: bool,       weather or not should continue to next
            result: parse_class,  the parsed result, if applicable
            num_fails: int         the number of failed attempts
        }
        """

        # input did't make it past the input condition for the edge
        if not self.check(input):
            self.num_fails += 1
            if self.num_fails >= self.max_retrys:
                return {"continue": True, "result": None, "num_fails": self.num_fails}
            return {"continue": False, "result": None, "num_fails": self.num_fails}

        try:
            # attempting to parse
            self.num_fails = 0
            return {
                "continue": True,
                "result": self.parse(input),
                "num_fails": self.num_fails,
            }
        except:
            # there was some error in parsing.
            # note, using the retry or correction parser here might be a good idea
            self.num_fails += 1
            if self.num_fails >= self.max_retrys:
                return {"continue": True, "result": None, "num_fails": self.num_fails}
            return {"continue": False, "result": None, "num_fails": self.num_fails}

In [41]:
# defining the query for the condition, and parse prompt
condition = "Does the input contain fruits?"
parse_prompt = "extract only the fruits from the following text. Do not extract any food items besides pure fruits."

In [42]:
# defining the edge
testEdge = Edge(
    condition=condition,
    parse_prompt=parse_prompt,
    parse_class=sampleOutputTemplate,
    llm=model,
)

In [43]:
# a sample input from the user
sample_input = "my favorite deserts are chocolate covered strawberries, oreos, bannana splits, and cake."

In [44]:
print("===== testing the condition functionality =====")
print(
    f'weather or not the input \n"{sample_input}"\nsatisfies the condition\n"{condition}"'
)
print("result: {}".format(testEdge.check(sample_input)))

===== testing the condition functionality =====
weather or not the input 
"my favorite deserts are chocolate covered strawberries, oreos, bannana splits, and cake."
satisfies the condition
"Does the input contain fruits?"
result: True


In [45]:
print("===== parse results =====")
print(testEdge.parse(sample_input).output)

===== parse results =====


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-ZZs7Rw1ObMQMMWL9FpbFQxMB on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

## Nodes

Now that we have an Edge, which handles input validation and parsing, we can define a Node, which handles conversational state. The Node requests a user for input, and passes that input to the directed edges coming from that Node. If none of the edges execute successfully, the Node asks the user for the input again.

In [ ]:
class Node:

    """Node
    at its highest level, a node asks a user for some input, and trys
    that input on all edges. It also manages and executes all
    the edges it contains
    """

    def __init__(self, prompt, retry_prompt):
        """
        prompt (str): what to ask the user
        retry_prompt (str): what to ask the user if all edges fail
        parse_class (Pydantic BaseModel): the structure of the parse
        llm (LangChain LLM): the large language model being used
        """

        self.prompt = prompt
        self.retry_prompt = retry_prompt
        self.edges = []

    def run_to_continue(self, _input):
        """Run all edges until one continues
        returns the result of the continuing edge, or None
        """
        for edge in self.edges:
            res = edge.execute(_input)
            if res["continue"]:
                return res
        return None

    def execute(self):
        """Handles the current conversational state
        prompots the user, tries again, runs edges, etc.
        returns the result from an adge
        """

        # initial prompt for the conversational state
        system_output(self.prompt)

        while True:
            # getting users input
            _input = user_input()

            # running through edges
            res = self.run_to_continue(_input)

            if res is not None:
                # parse successful
                parsing_info(f"parse results: {res}")
                return res

            # unsuccessful, prompting retry
            system_output(self.retry_prompt)

With this implemented, we can begin seeing conversations take place. We’ll implement a Node which requests contact information, and two edges: one which attempts to parse out a valid email, and one that attempts to parse out a valid phone number.

## Connecting Nodes and Edges

In [ ]:
# Defining 2 edges from the node

condition1 = "Does the input contain a full and valid email?"
parse_prompt1 = "extract the email from the following text."
edge1 = Edge(condition1, parse_prompt1, sampleOutputTemplate, model)
condition2 = (
    "Does the input contain a full and valid phone number (xxx-xxx-xxxx or xxxxxxxxxx)?"
)
parse_prompt2 = "extract the phone number from the following text."
edge2 = Edge(condition2, parse_prompt2, sampleOutputTemplate, model)

In [ ]:
# Defining A Node
test_node = Node(
    prompt="Please input your full email address or phone number",
    retry_prompt="I'm sorry, I didn't understand your response.\nPlease provide a full email address or phone number(in the format xxx-xxx-xxxx)",
)

In [ ]:
# Defining Connections
test_node.edges = [edge1, edge2]

In [ ]:
# running node. This handles all i/o and the logic to re-ask on failure.
res = test_node.execute()

# Customer Support flow

In [3]:
from data.chat import *
from data.graph import *
from data.validation import PhoneCallTicket, UserProfile, PhoneCallRequest
from graph.chain_based_edge import *
from graph.chain_based_node import *
from graph.edge import BaseEdge
from graph.node import BaseNode
from graph.text_based_edge import PydanticTextBasedEdge
from tools.rag_responder import HelpCenterAgent
from tools.user_info_db import search_user_info_on_db, search_user_subscription_on_db
from tools.audio_transcribe import call_customer

In [4]:
llm_model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [5]:
def get_message_history(msg_input: str, role: Role):
    message_history = MessageHistory([])
    message_history.add_message(msg_input, role)
    return message_history

In [6]:
class GreetingNode(BaseNode[str]):
    STATIC_PROMPT = [
        "Hi, welcome to our online support, in order to proceed we need to identify you first, "
        "could you please input your full email address or phone number"
    ]
    RETRY_PROMPT = [
        "I'm sorry, I didn't understand your response."
        "\nPlease provide a full email address or phone number(in the format xxx-xxx-xxxx)"
    ]

    def greeting_message(self) -> Optional[MessageOutput]:
        prompt = random.choice(self.STATIC_PROMPT)
        return MessageOutput(prompt, role=Role.ASSISTANT)

    def no_edges_found(self, **kwargs) -> Optional[MessageOutput]:
        prompt = random.choice(self.RETRY_PROMPT)
        return MessageOutput(prompt, role=Role.ASSISTANT)

In [7]:
print(inspect.getsource(BaseNode))

class BaseNode(abc.ABC, Generic[NodeInput]):

    """Node
    at it's highest level, a node asks a user for some input, and trys
    that input on all edges. It also manages and executes all
    the edges it contains
    """

    def __init__(self, edges: Optional[List[BaseEdge]] = None, final_state=False):
        """
        prompt (str): what to ask the user
        retry_prompt (str): what to ask the user if all edges fail
        parse_class (Pydantic BaseModel): the structure of the parse
        llm (LangChain LLM): the large language model being used
        """

        self._edges = edges
        self._node_input = None
        self._final_state = final_state

    def is_node_final(self):
        return self._final_state

    def set_node_input(self, edge_output: EdgeOutput):
        self._node_input = edge_output

    def run_to_continue(self, user_input: NodeInput) -> Optional[EdgeOutput]:
        """Run all edges until one continues
        returns the result of the contin

In [8]:
greeting_node = GreetingNode()

In [9]:
greeting_node.greeting_message()

MessageOutput(message='Hi, welcome to our online support, in order to proceed we need to identify you first, could you please input your full email address or phone number', role=<Role.ASSISTANT: 'assistant'>)

In [10]:
class UserInfoChainBasedEdge(ZeroShotChainBasedEdge):
    _prompt_prefix = """Your goal is to find out the user information and their subscription type.
- You MUST ALWAYS combine the output of different tools to achieve your final answer.
- The user subscription must be either free or premium, never empty

To achieve this you have access to the following tools:"""

    _prompt_suffix = """\nYour final answer should combine the information of previous Observations 
{format_instructions}
Begin! 
Question: {input}
{agent_scratchpad}
"""

    def _get_tools(self):
        tools = [
            Tool.from_function(
                func=search_user_info_on_db,
                description="Database tool to search user information, like user id, phone number and etc.Input should be their email as text",
                name="user_info_db_search",
            ),
            Tool.from_function(
                func=search_user_subscription_on_db,
                description="Database tool to search user subscription type by user id, requires a number as input,",
                name="user_subscription_db_search",
            ),
        ]
        return tools

    def _get_message_output(
        self, msg_input: Union[str, BaseModel]
    ) -> List[MessageOutput]:
        user_info = msg_input if isinstance(msg_input, str) else str(msg_input)
        message = f"User Info retrieved: {user_info}"
        return [MessageOutput]

In [11]:
print(inspect.getsource(BaseEdge))

class BaseEdge(abc.ABC, Generic[EdgeInput, ResultsType]):
    def __init__(self, model, max_retries=3, out_node=None):
        self._llm_model = model

        # how many times the edge has failed, for any reason, for deciding to skip
        # when successful this resets to 0 for posterity.
        self._num_fails = 0

        # how many retrys are acceptable
        self._max_retries = max_retries

        # the node the edge directs towards
        self._out_node = out_node

    @abc.abstractmethod
    def _get_message_output(
        self, msg_input: Union[str, BaseModel]
    ) -> Optional[List[MessageOutput]]:
        pass

    @abc.abstractmethod
    def check(self, model_output: str) -> bool:
        pass

    @abc.abstractmethod
    def _parse(self, model_input: EdgeInput) -> ResultsType:
        pass

    def _get_edge_output(
        self, should_continue: bool, result: Optional[ResultsType]
    ) -> EdgeOutput:
        message_output = self._get_message_output(result)
      

In [12]:
print(inspect.getsource(ChainBasedEdge))

class ChainBasedEdge(BaseEdge[MessageHistory, MessageOutput], ABC):
    def __init__(
        self,
        model,
        pydantic_object: Optional[Type[BaseModel]],
        max_retries=3,
        out_node=None,
    ):
        super().__init__(model=model, max_retries=max_retries, out_node=out_node)
        if pydantic_object is not None:
            self._output_parser = PydanticOutputParser(pydantic_object=pydantic_object)
        else:
            self._output_parser = None

        self._init_chain()

    @abc.abstractmethod
    def _predict(self, model_input: ModelInput) -> str:
        pass

    @abc.abstractmethod
    def _init_chain(self, *kwargs):
        pass

    @abc.abstractmethod
    def _get_prompt_template(self) -> BasePromptTemplate:
        pass

    @abc.abstractmethod
    def _prompt_input_variables(self) -> list:
        pass

    def check(self, model_output: str) -> bool:
        return isinstance(self._output_parser.parse(model_output), BaseModel)

    def _par

In [13]:
print(inspect.getsource(ZeroShotChainBasedEdge))

class ZeroShotChainBasedEdge(ChainBasedEdge, ABC):
    _prompt_prefix = None
    _prompt_suffix = None

    def _prompt_input_variables(self):
        input_variables = ["input", "agent_scratchpad", "history"]
        if self._output_parser is not None:
            input_variables += ["format_instructions"]

        return input_variables

    def _get_prompt_template(self) -> BasePromptTemplate:
        input_variables = self._prompt_input_variables()
        history = """Conversation History \n{history}\n"""

        prompt = ZeroShotAgent.create_prompt(
            tools=self._tools,
            prefix=self._prompt_prefix,
            suffix=history + self._prompt_suffix,
            input_variables=input_variables,
        )
        return prompt

    def _init_chain(self, **kwargs):
        self._tools = self._get_tools()

        self._prompt = self._get_prompt_template()
        self._llm_chain = LLMChain(llm=self._llm_model, prompt=self._prompt)

        agent = ZeroShotAgent(l

In [14]:
user_info_edge = UserInfoChainBasedEdge(model=llm_model, pydantic_object=UserProfile)

In [15]:
user_info_edge.execute(get_message_history("michaeljackson@gmail.com", Role.USER))



> Entering new AgentExecutor chain...
Thought: I need to find the user information and their subscription type.
Action: user_info_db_search
Action Input: michaeljackson@gmail.com
Observation: [{'name': 'Michael Jackson', 'email': 'michaeljackson@gmail.com', 'user_id': '1', 'phone': '0452 333 666', 'language': 'English'}]
Thought:I have found the user information for michaeljackson@gmail.com. Now I need to find their subscription type.
Action: user_subscription_db_search
Action Input: 1
Observation: [{'user_id': '1', 'subscription': 'premium'}]
Thought:I now know the final answer.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Question: What is the user information and their subscription type for michaeljackson@gmail.com?
Thought: I need to find the user information and their subscription type.
Action: user_info_db_search
Action Input: michaeljackson@gmail.com
Observation: [{'name': 'Michael Jackson', 'email': 'michaeljackson@gmail.com', 'user_id': '1', 'phone'

EdgeOutput(should_continue=True, result=UserProfile(name='Michael Jackson', email='michaeljackson@gmail.com', subscription='premium', user_id=1, phone='0452 333 666', language='English'), message_output=[<class 'data.graph.MessageOutput'>], num_fails=0, next_node=None)

In [16]:
class AuthenticatedUserNode(MultiRetrievalNode):
    STATIC_PROMPT = [
        "Hi, {user_name} I am your Shopify Agent for today, you have the {subscription} subscription "
        "I can help you with any Help or you can ask me to call you at anytime!"
    ]

    def __init__(
        self,
        llm_model,
        pydantic_object: Optional[Type[BaseNode]],
        edges: List[BaseEdge] = None,
    ):
        self._hc_agent = HelpCenterAgent()
        super().__init__(llm_model, pydantic_object, edges)

    def greeting_message(self) -> Optional[MessageOutput]:
        prompt = random.choice(self.STATIC_PROMPT)
        user_profile: UserProfile = self._node_input

        prompt = prompt.format(
            user_name=user_profile.name, subscription=user_profile.subscription
        )
        return MessageOutput(prompt, role=Role.ASSISTANT)

    def _get_retriever_infos(self):
        retriever_infos = [
            {
                "name": "Premium Subscription Knowledge Base",
                "description": "Contains information for user with a premium subscription",
                "retriever": self._hc_agent.paid_sub_retriever(),
            },
            {
                "name": "Free Subscription Knowledge Base",
                "description": "Contains information for user with a free subscription",
                "retriever": self._hc_agent.free_sub_retriever(),
            },
        ]
        return retriever_infos

    def _get_default_chain(self):
        template = """You are a helpful assistant, you should tell the user that his query is outside of your domain 
    in a friendly way"
    Human: """

        prompt_template = PromptTemplate.from_template(template)
        chain = LLMChain(
            llm=self._llm_model, prompt=prompt_template, output_key="result"
        )
        return chain

    def no_edges_found(self, user_input: MessageHistory) -> Optional[MessageOutput]:
        message = self._predict(user_input)
        return MessageOutput(message=message, role=Role.ASSISTANT)

In [17]:
print(inspect.getsource(ChainBasedNode))

class ChainBasedNode(BaseNode[MessageHistory], abc.ABC):
    def __init__(
        self,
        llm_model,
        pydantic_object: Optional[Type[BaseModel]],
        edges: Optional[List[BaseEdge]],
        final_state=False,
    ):
        self._llm_model = llm_model
        self._parse_class = pydantic_object

        if pydantic_object is not None:
            self._output_parser = PydanticOutputParser(pydantic_object=pydantic_object)
        else:
            self._output_parser = None

        self._init_chain()
        super().__init__(edges, final_state)

    @abc.abstractmethod
    def _init_chain(self, **kwargs):
        pass



In [18]:
print(inspect.getsource(MultiRetrievalNode))

class MultiRetrievalNode(ChainBasedNode, abc.ABC):
    @abc.abstractmethod
    def _get_retriever_infos(self):
        pass

    @abc.abstractmethod
    def _get_default_chain(self):
        pass

    def _init_chain(self, *kwargs):
        retriever_infos = self._get_retriever_infos()

        self._llm_chain = MultiRetrievalQAChain.from_retrievers(
            self._llm_model,
            retriever_infos,
            default_chain=self._get_default_chain(),
            verbose=True,
        )

    def _predict(self, messages: MessageHistory) -> str:
        return self._llm_chain.run(messages)



In [19]:
authenticated_user_node = AuthenticatedUserNode(
    llm_model=llm_model, pydantic_object=None, edges=[]
)

In [20]:
authenticated_user_node.execute(
    get_message_history("User with Premium Subscription: What is a POS", Role.USER)
)



> Entering new MultiRetrievalQAChain chain...


/Users/rpossas/miniconda3/envs/tensorflow/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Premium Subscription Knowledge Base: {'query': 'What is a POS'}
> Finished chain.


MessageOutput(message="A POS, or point of sale, refers to the location where a transaction takes place. It can also refer to the software or system used to process sales transactions. In the context of Shopify, Shopify POS is a point of sale app that allows merchants to sell their products in person, whether it's in retail stores, pop-up shops, or other locations. The Shopify POS app is available for iOS and Android devices and syncs with Shopify to track orders and inventory across different sales channels.", role=<Role.ASSISTANT: 'assistant'>)